From: https://elnamagnetics.com/wp-content/uploads/library/Ferroxcube-Documents/Design_of_Planar_Power_Transformers.pdf

Core datasheet: https://www.farnell.com/datasheets/1524995.pdf

TI Paper: https://www.ti.com/download/trng/docs/seminar/Topic4LD.pdf



In [ ]:
from math import sqrt, pi, asin, log

# Design constraints, Lab 6
Vg = 18                       # Input voltage, V
Vo = 10                       # Output voltage, V
Pout = 20                     # Output power, W
fs = 50e3                     # Switching frequency, fs
Ts = 1/fs                     # Switching period
D = 0.35                      # Duty cycle
alpha = 0.8                   # Critical inductance ratio
Bmax = 0.3                    # Maximum flux density, T
a = Vo/Vg*(sqrt(alpha)-D)/D   # Turns ratio
L = 18e-6                     # Desired inductance, H
Ro = 5                        # Output load resistance, Ohms

uo = pi*4e-7                  # Permeability of free space, N/A^2

# Lab hardware dimensions
X1w = 23.2e-3                 # transformer footprint width, m
X1h = 23.2e-3                 # transformer footprint length, m

# Transformer calculations
# Using core E18
Ae = 39.5e-6                  # effective cross sectional area, m^2

N1_calc = (Vg*D)/(2*fs*Bmax*Ae)    # Primary turns
N2_calc = (N1_calc*Vo)/Vg          # Secondary turns

N1 = 5
N2 = 3

Lprim = (Vg*D)**2/(2*Pout*fs) # Primary winding inductance, H

Le = (uo*N1**2*Ae)/Lprim      # Airgap length, in m
print(Le)
Le_thou = Le*39.37e3          # Meters -> thousandths of inch

print("TRANSFORMER SPECS \n Primary turns:", round(N1, 2),
      "\n Secondary turns:", round(N2, 2),
      "\n Primary inductance:", round(Lprim*1e6, 2), "uH",
      "\n Calculated air gap", round(Le_thou, 2), "thou \n")

Io = (Pout/Vo)*sqrt(4/(3*D))     # secondary output current (RMS), A
Ip = (Vg*D)/(fs*Lprim)*sqrt(D/3) # primary input current (RMS), A

print("RMS CURRENTS \n RMS input current:", round(Io, 2),
      "\n RMS output current:", round(Ip, 2), "\n")

# E-18 core specs
bw = 3.6                        # Winding width, mm
s = 0.3                         # Spacing between tracks, mm
wt_calc = (bw - 0.8 - (N1-1)*s)/N1   # Track width, mm
wt_m_calc = wt_calc/1000

print("E-18 Calculated Track width:", wt_m_calc*1e3, "mm")

wt_m = 0.52/1000 # using track width of 0.52 mm

# temperature rise based on track width
winding_t = 70e-6            # thickness of winding layer, in m
track_cross_section = wt_m*winding_t
print("cross section of track:",track_cross_section*1e6,"mm^2")

# pcb stackup calcs:
ins_t = 200e-6               # thickness of insulation, in m
smask_t = 50e-6              # solder mask thickness, in m
pcb_stackup = ins_t*3 + smask_t*2 + winding_t*4
print("Total PCB stackup:", pcb_stackup, "m")

6.253107070637282e-05
TRANSFORMER SPECS 
 Primary turns: 5 
 Secondary turns: 3 
 Primary inductance: 19.84 uH 
 Calculated air gap 2.46 thou 

RMS CURRENTS 
 RMS input current: 3.9 
 RMS output current: 2.17 

E-18 Calculated Track width: 0.31999999999999995 mm
cross section of track: 0.0364 mm^2
Total PCB stackup: 0.00098 m


In [ ]:
# Leakage inductance calc using TI paper
H = (N1*Ip)/bw                          # Field intensity
E_density = 0.5*uo*H**2                 # Energy density, joules/m^3
v = winding_t*(bw*1e-3)*(bw*1e-3)       # Volume of inter-winding region
E_tot = E_density*v                     # Total energy
L_sigma = (2*E_tot)/Ip**2

print(L_sigma)

2.1991148575128555e-15


In [ ]:
# Parasitic capacitance
er = 4.8          # Permittivity of dielectric between layers (FR4), F/m
e0 = 8.854e-12    # Permitivity of free space, F/m
d = 0.2           # distance between layers,m
n = 5             # Num. of turns between both layers (same for both top-middle & middle-bottom pairs)
m = 2             # Num. layers in series

# interwinding capacitance of primary
A = 81.8e-3*0.52e-3                 # Area of trace
C0 = er*e0*A/d                      # Capacitance of effective area
Cd = ((n+1)*(2*n+1))/(12*n)*C0      # Capacitance between two layers with n turns
Cw = (4*(m-1))/m**2*Cd              # Overall capacitance, m layers in series for the same winding
print("Interwinding capacitance of primary winding:", Cw)

C_total = Cd+Cw

print("Total parasitic capacitance:", C_total, "\nTotal parasitic capacitance (pF):", C_total*1e12) # 0.0198 pF



Interwinding capacitance of primary winding: 9.942602841599997e-15
Total parasitic capacitance: 1.9885205683199995e-14 
Total parasitic capacitance (pF): 0.019885205683199996
